In [ ]:
import re
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
class ProjectScheduler:
    def __init__(self, file_path):
        self.data = {
            'projects': 0,
            'jobs': 0,
            'horizon': 0,
            'resources': {'renewable': 0, 'nonrenewable': 0},
            'precedence_relations': {},
            'job_details': {},
            'resource_availability': {}
        }
        self.parse_file(file_path)
    
    def parse_file(self, file_path):
        with open(file_path, 'r') as file:
            content = file.read()
        
        # Parse general information
        self.data['projects'] = int(re.search(r'projects:\s*(\d+)', content).group(1))
        self.data['jobs'] = int(re.search(r'jobs \(incl\. supersource/sink \):\s*(\d+)', content).group(1))
        self.data['horizon'] = int(re.search(r'horizon:\s*(\d+)', content).group(1))
        
        # Parse resources
        resources_match = re.search(r'RESOURCES\n  - renewable\s*:\s*(\d+)\s*R\n  - nonrenewable\s*:\s*(\d+)\s*N', content)
        self.data['resources']['renewable'] = int(resources_match.group(1))
        self.data['resources']['nonrenewable'] = int(resources_match.group(2))
        
        # Parse precedence relations
        precedence_section = re.search(r'#Precedence relations(.*?)#Duration', content, re.DOTALL)
        if precedence_section:
            for line in precedence_section.group(1).strip().split('\n')[1:]:
                parts = line.split()
                if len(parts) > 2:
                    job_nr = int(parts[0])
                    successors = [int(succ) for succ in parts[2:]]
                    self.data['precedence_relations'][job_nr] = successors
        
        # Parse job details
        duration_section = re.search(r'#Duration and resources(.*?)#Resource', content, re.DOTALL)
        if duration_section:
            for line in duration_section.group(1).strip().split('\n')[1:]:
                parts = line.split()
                if len(parts) > 2:
                    job_nr = int(parts[0])
                    duration = int(parts[2])
                    resources = {f'R{i+1}': int(parts[3+i]) for i in range(2)}
                    self.data['job_details'][job_nr] = {
                        'duration': duration,
                        'resources': resources
                    }
        
        # Parse resource availability
        availability_section = re.search(r'#Resource availability(.*)', content, re.DOTALL)
        if availability_section:
            for line in availability_section.group(1).strip().split('\n')[1:]:
                parts = line.split()
                if len(parts) > 1:
                    resource = parts[0]
                    quantity = int(parts[1])
                    self.data['resource_availability'][resource] = quantity
    
    def get_project_summary(self):
        return {
            'Total Projects': self.data['projects'],
            'Total Jobs': self.data['jobs'],
            'Time Horizon': self.data['horizon'],
            'Renewable Resources': self.data['resources']['renewable'],
            'Resource Availability': self.data['resource_availability']
        }
    
    def get_job_precedence(self):
        return self.data['precedence_relations']
    
    def get_job_details(self):
        return self.data['job_details']

In [ ]:
def create_project_graph(precedence_relations):
    # Cria um grafo direcionado
    G = nx.DiGraph()
    
    # Adiciona arestas baseadas nas relações de precedência
    for job, successors in precedence_relations.items():
        for successor in successors:
            G.add_edge(job, successor)
    
    # Configura o layout do grafo
    pos = nx.spring_layout(G, k=0.5, iterations=50)
    
    # Desenha o grafo
    plt.figure(figsize=(10, 6))
    nx.draw(G, pos, with_labels=True, 
            node_color='lightblue', 
            node_size=500, 
            arrows=True, 
            edge_color='gray')
    
    # Adiciona título
    plt.title("Grafo de Precedência de Tarefas")
    
    # Mostra o grafo
    plt.show()